# Full Net Tracking Analysis: Multi-System Performance Assessment

This notebook performs in-depth analysis of the synchronized comparison data from three net-tracking systems:
- **FFT-based method**: Frequency domain signal processing
- **Sonar image tracker**: Computer vision on multibeam sonar
- **DVL reference**: Doppler Velocity Log baseline

**Analysis Goals:**
1. Identify stable baseline segments (clean data periods)
2. Quantify noise and bias in each method
3. Detect and characterize outliers/spikes
4. Analyze failure correlation between methods
5. Handle DVL closed-loop control effects

**Note:** This assumes timestamps are pre-aligned from the comparison CSV.

## 1. Setup and Configuration

In [196]:
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import signal, stats
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# Configuration
TARGET_BAG = "2024-08-20_13-57-42"  # Change to your bag ID
COMPARISON_DATA_DIR = Path("/Volumes/LaCie/SOLAQUA/comparison_data")
PLOTS_DIR = Path("/Volumes/LaCie/SOLAQUA/exports/plots")
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

# Smoothing configuration (applied to raw data before analysis)
SMOOTHING_ALPHA = None  # Exponential smoothing factor (0-1), None = no smoothing
# Examples:
#   SMOOTHING_ALPHA = None   # No smoothing (raw data)
#   SMOOTHING_ALPHA = 0.1    # Heavy smoothing (90% history, 10% current)
#   SMOOTHING_ALPHA = 0.3    # Moderate smoothing (70% history, 30% current)
#   SMOOTHING_ALPHA = 0.5    # Balanced smoothing (50% history, 50% current)
#   SMOOTHING_ALPHA = 0.7    # Light smoothing (30% history, 70% current)
#   SMOOTHING_ALPHA = 0.9    # Very light smoothing (10% history, 90% current)

# Analysis parameters
ROLLING_WINDOW_SEC = 5.0  # Window for rolling statistics (seconds)
SIGMA_THRESH = 0.15  # Stability threshold for std deviation (meters)
DELTA_THRESH = 0.15  # Stability threshold for mean difference (meters)
MIN_SEGMENT_SEC = 1.0  # Minimum baseline segment duration (seconds)
OUTLIER_K = 3.5  # MAD multiplier for outlier detection

print(f"Analyzing: {TARGET_BAG}")
print(f"Data source: {COMPARISON_DATA_DIR}")
print(f"Plots: {PLOTS_DIR}\n")
print("Configuration:")
if SMOOTHING_ALPHA is not None and SMOOTHING_ALPHA < 1.0:
    print(f"  Smoothing: Enabled (α={SMOOTHING_ALPHA:.2f})")
else:
    print(f"  Smoothing: Disabled (raw data)")
print(f"  Rolling window: {ROLLING_WINDOW_SEC} s")
print(f"  Stability σ threshold: {SIGMA_THRESH} m")
print(f"  Stability Δ threshold: {DELTA_THRESH} m")
print(f"  Min segment length: {MIN_SEGMENT_SEC} s")
print(f"  Outlier threshold: {OUTLIER_K} × MAD")

Analyzing: 2024-08-20_13-57-42
Data source: /Volumes/LaCie/SOLAQUA/comparison_data
Plots: /Volumes/LaCie/SOLAQUA/exports/plots

Configuration:
  Smoothing: Disabled (raw data)
  Rolling window: 5.0 s
  Stability σ threshold: 0.15 m
  Stability Δ threshold: 0.15 m
  Min segment length: 1.0 s
  Outlier threshold: 3.5 × MAD


## 2. Load and Prepare Data

In [197]:
# Load comparison data
data_path = COMPARISON_DATA_DIR / f"{TARGET_BAG}_raw_comparison.csv"

if not data_path.exists():
    raise FileNotFoundError(f"Comparison data not found: {data_path}")

print(f"Loading data from: {data_path.name}")
df = pd.read_csv(data_path)

# Parse timestamp
df['timestamp'] = pd.to_datetime(df['sync_timestamp'])
df = df.set_index('timestamp').sort_index()

# Apply smoothing if specified
if SMOOTHING_ALPHA is not None and SMOOTHING_ALPHA < 1.0:
    print(f"\nApplying exponential smoothing (α={SMOOTHING_ALPHA:.2f})...")
    
    # Columns to smooth
    smooth_cols = [
        'fft_distance_m', 'fft_pitch_deg', 'fft_x', 'fft_y',
        'sonar_distance_m', 'sonar_pitch_deg', 'sonar_x', 'sonar_y',
        'nav_distance_m', 'nav_pitch_deg', 'dvl_x', 'dvl_y'
    ]
    
    for col in smooth_cols:
        if col in df.columns:
            df[col] = df[col].ewm(alpha=SMOOTHING_ALPHA, adjust=False).mean()
    
    print(f"  ✓ Smoothed {sum(1 for c in smooth_cols if c in df.columns)} columns")

# Calculate sampling rate
time_diffs = df.index.to_series().diff().dt.total_seconds()
median_dt = time_diffs.median()
sampling_rate = 1.0 / median_dt if median_dt > 0 else None

print(f"\nData loaded: {len(df)} samples")
print(f"Time range: {df.index[0]} to {df.index[-1]}")
print(f"Duration: {(df.index[-1] - df.index[0]).total_seconds():.1f} seconds")
print(f"Median sampling interval: {median_dt:.3f} s ({sampling_rate:.1f} Hz)")

# Check for available systems
available_systems = []
if 'fft_distance_m' in df.columns and df['fft_distance_m'].notna().any():
    available_systems.append('FFT')
if 'sonar_distance_m' in df.columns and df['sonar_distance_m'].notna().any():
    available_systems.append('Sonar')
if 'nav_distance_m' in df.columns and df['nav_distance_m'].notna().any():
    available_systems.append('DVL')

print(f"Available systems: {', '.join(available_systems)}")

# Display sample
print("\nData sample:")
display_cols = [c for c in ['fft_distance_m', 'sonar_distance_m', 'nav_distance_m', 
                             'fft_pitch_deg', 'sonar_pitch_deg', 'nav_pitch_deg'] 
                if c in df.columns]
print(df[display_cols].head())

Loading data from: 2024-08-20_13-57-42_raw_comparison.csv

Data loaded: 2744 samples
Time range: 2024-08-20 11:57:45.106798649+00:00 to 2024-08-20 11:59:04.976633549+00:00
Duration: 79.9 seconds
Median sampling interval: 0.025 s (39.2 Hz)
Available systems: FFT, Sonar, DVL

Data sample:
                                     fft_distance_m  sonar_distance_m  \
timestamp                                                               
2024-08-20 11:57:45.106798649+00:00        1.273458          1.355712   
2024-08-20 11:57:45.137919903+00:00        1.273458          1.355712   
2024-08-20 11:57:45.170873165+00:00        1.273458          1.355712   
2024-08-20 11:57:45.182058334+00:00        1.273458          1.355712   
2024-08-20 11:57:45.214040756+00:00        1.273458          1.355712   

                                     nav_distance_m  fft_pitch_deg  \
timestamp                                                            
2024-08-20 11:57:45.106798649+00:00             NaN     -12.

## 3. Compute Pairwise Differences

In [198]:
# Compute pairwise differences for distance
if 'fft_distance_m' in df.columns and 'nav_distance_m' in df.columns:
    df['diff_fft_nav'] = df['fft_distance_m'] - df['nav_distance_m']
    
if 'sonar_distance_m' in df.columns and 'nav_distance_m' in df.columns:
    df['diff_sonar_nav'] = df['sonar_distance_m'] - df['nav_distance_m']
    
if 'fft_distance_m' in df.columns and 'sonar_distance_m' in df.columns:
    df['diff_fft_sonar'] = df['fft_distance_m'] - df['sonar_distance_m']

# Compute pairwise differences for pitch
if 'fft_pitch_deg' in df.columns and 'nav_pitch_deg' in df.columns:
    df['diff_pitch_fft_nav'] = df['fft_pitch_deg'] - df['nav_pitch_deg']
    
if 'sonar_pitch_deg' in df.columns and 'nav_pitch_deg' in df.columns:
    df['diff_pitch_sonar_nav'] = df['sonar_pitch_deg'] - df['nav_pitch_deg']

if 'fft_pitch_deg' in df.columns and 'sonar_pitch_deg' in df.columns:
    df['diff_pitch_fft_sonar'] = df['fft_pitch_deg'] - df['sonar_pitch_deg']

# Compute pairwise differences for X position
if 'fft_x' in df.columns and 'dvl_x' in df.columns:
    df['diff_x_fft_nav'] = df['fft_x'] - df['dvl_x']
    
if 'sonar_x' in df.columns and 'dvl_x' in df.columns:
    df['diff_x_sonar_nav'] = df['sonar_x'] - df['dvl_x']

if 'fft_x' in df.columns and 'sonar_x' in df.columns:
    df['diff_x_fft_sonar'] = df['fft_x'] - df['sonar_x']

# Compute pairwise differences for Y position
if 'fft_y' in df.columns and 'dvl_y' in df.columns:
    df['diff_y_fft_nav'] = df['fft_y'] - df['dvl_y']
    
if 'sonar_y' in df.columns and 'dvl_y' in df.columns:
    df['diff_y_sonar_nav'] = df['sonar_y'] - df['dvl_y']

if 'fft_y' in df.columns and 'sonar_y' in df.columns:
    df['diff_y_fft_sonar'] = df['fft_y'] - df['sonar_y']

print("Pairwise differences computed:")
diff_cols = [c for c in df.columns if c.startswith('diff_')]
for col in diff_cols:
    valid_count = df[col].notna().sum()
    if valid_count > 0:
        print(f"  {col}: {valid_count} valid samples, "
              f"mean={df[col].mean():.3f}, std={df[col].std():.3f}")

Pairwise differences computed:
  diff_fft_nav: 2704 valid samples, mean=-0.133, std=1.176
  diff_sonar_nav: 2704 valid samples, mean=-0.021, std=0.169
  diff_fft_sonar: 2744 valid samples, mean=-0.110, std=1.148
  diff_pitch_fft_nav: 2704 valid samples, mean=-1.633, std=21.451
  diff_pitch_sonar_nav: 2704 valid samples, mean=-1.389, std=8.883
  diff_pitch_fft_sonar: 2744 valid samples, mean=-0.085, std=21.371
  diff_x_fft_nav: 2704 valid samples, mean=-0.145, std=1.167
  diff_x_sonar_nav: 2704 valid samples, mean=-0.022, std=0.185
  diff_x_fft_sonar: 2744 valid samples, mean=-0.120, std=1.136
  diff_y_fft_nav: 2704 valid samples, mean=-0.003, std=0.430
  diff_y_sonar_nav: 2704 valid samples, mean=-0.037, std=0.212
  diff_y_fft_sonar: 2744 valid samples, mean=0.037, std=0.430


## 4. Detect Stable Baseline Segments

Identify time windows where all methods show:
- Low variance (stable measurements)
- Small mutual differences (good agreement)
- No missing data

In [199]:
# Calculate rolling window size in samples
window_samples = int(ROLLING_WINDOW_SEC * sampling_rate) if sampling_rate else 30
print(f"Rolling window: {window_samples} samples ({ROLLING_WINDOW_SEC} s)\n")

# Compute rolling statistics
print("Computing rolling statistics...")

# Rolling std for each method
if 'fft_distance_m' in df.columns:
    df['rolling_std_fft'] = df['fft_distance_m'].rolling(window_samples, center=True).std()
if 'sonar_distance_m' in df.columns:
    df['rolling_std_sonar'] = df['sonar_distance_m'].rolling(window_samples, center=True).std()
if 'nav_distance_m' in df.columns:
    df['rolling_std_dvl'] = df['nav_distance_m'].rolling(window_samples, center=True).std()

# Rolling mean and std of differences
if 'diff_fft_nav' in df.columns:
    df['rolling_mean_diff_fft_nav'] = df['diff_fft_nav'].rolling(window_samples, center=True).mean()
    df['rolling_std_diff_fft_nav'] = df['diff_fft_nav'].rolling(window_samples, center=True).std()
    
if 'diff_sonar_nav' in df.columns:
    df['rolling_mean_diff_sonar_nav'] = df['diff_sonar_nav'].rolling(window_samples, center=True).mean()
    df['rolling_std_diff_sonar_nav'] = df['diff_sonar_nav'].rolling(window_samples, center=True).std()

# Define stability criteria
print("Applying stability criteria...")
stable_mask = pd.Series(True, index=df.index)

# Low variance in each method
if 'rolling_std_fft' in df.columns:
    stable_mask &= (df['rolling_std_fft'] < SIGMA_THRESH)
if 'rolling_std_sonar' in df.columns:
    stable_mask &= (df['rolling_std_sonar'] < SIGMA_THRESH)
if 'rolling_std_dvl' in df.columns:
    stable_mask &= (df['rolling_std_dvl'] < SIGMA_THRESH)

# Small mutual differences
if 'rolling_mean_diff_fft_nav' in df.columns:
    stable_mask &= (df['rolling_mean_diff_fft_nav'].abs() < DELTA_THRESH)
if 'rolling_mean_diff_sonar_nav' in df.columns:
    stable_mask &= (df['rolling_mean_diff_sonar_nav'].abs() < DELTA_THRESH)

# No NaNs in distance measurements
for col in ['fft_distance_m', 'sonar_distance_m', 'nav_distance_m']:
    if col in df.columns:
        stable_mask &= df[col].notna()

df['is_stable'] = stable_mask

print(f"Stable samples: {stable_mask.sum()} / {len(df)} ({100*stable_mask.sum()/len(df):.1f}%)")

Rolling window: 196 samples (5.0 s)

Computing rolling statistics...
Applying stability criteria...
Stable samples: 776 / 2744 (28.3%)


In [200]:
# Identify connected stable segments
print("\nIdentifying stable baseline segments...")

# Find segment boundaries
stable_changes = df['is_stable'].astype(int).diff()
segment_starts = df.index[stable_changes == 1]
segment_ends = df.index[stable_changes == -1]

# Handle edge cases
if df['is_stable'].iloc[0]:
    segment_starts = pd.DatetimeIndex([df.index[0]]).append(segment_starts)
if df['is_stable'].iloc[-1]:
    segment_ends = segment_ends.append(pd.DatetimeIndex([df.index[-1]]))

# Create segment list
segments = []
min_duration = pd.Timedelta(seconds=MIN_SEGMENT_SEC)

for start, end in zip(segment_starts, segment_ends):
    duration = end - start
    if duration >= min_duration:
        segment_data = df.loc[start:end]
        segments.append({
            'start': start,
            'end': end,
            'duration_sec': duration.total_seconds(),
            'n_samples': len(segment_data),
            'mean_dvl_dist': segment_data['nav_distance_m'].mean() if 'nav_distance_m' in df.columns else np.nan
        })

print(f"Found {len(segments)} stable baseline segments (≥ {MIN_SEGMENT_SEC} s):\n")
for i, seg in enumerate(segments[:10]):  # Show first 10
    print(f"  Segment {i+1}: {seg['start'].strftime('%H:%M:%S')} - {seg['end'].strftime('%H:%M:%S')} "
          f"({seg['duration_sec']:.1f} s, {seg['n_samples']} samples)")
if len(segments) > 10:
    print(f"  ... and {len(segments)-10} more segments")

# Add segment ID to dataframe
df['segment_id'] = -1  # -1 = not in baseline
for i, seg in enumerate(segments):
    df.loc[seg['start']:seg['end'], 'segment_id'] = i


Identifying stable baseline segments...
Found 3 stable baseline segments (≥ 1.0 s):

  Segment 1: 11:57:59 - 11:58:09 (10.4 s, 363 samples)
  Segment 2: 11:58:23 - 11:58:25 (2.1 s, 73 samples)
  Segment 3: 11:58:51 - 11:59:01 (10.2 s, 343 samples)


## 5. Baseline Statistics: Noise and Bias

Quantify performance in stable regions only.

In [201]:
# Extract baseline samples
baseline_df = df[df['is_stable']].copy()

print(f"=== BASELINE STATISTICS ({len(baseline_df)} samples) ===\n")

# Per-method noise level - DISTANCE
print("1. PER-METHOD NOISE - DISTANCE (Standard Deviation in Baseline)")
for method, col in [('FFT', 'fft_distance_m'), ('Sonar', 'sonar_distance_m'), ('DVL', 'nav_distance_m')]:
    if col in baseline_df.columns:
        data = baseline_df[col].dropna()
        if len(data) > 0:
            std = data.std()
            mad = stats.median_abs_deviation(data, scale='normal')
            print(f"  {method}:")
            print(f"    σ (std dev) = {std:.4f} m")
            print(f"    MAD = {mad:.4f} m")

# Per-method noise level - PITCH
print("\n2. PER-METHOD NOISE - PITCH (Standard Deviation in Baseline)")
for method, col in [('FFT', 'fft_pitch_deg'), ('Sonar', 'sonar_pitch_deg'), ('DVL', 'nav_pitch_deg')]:
    if col in baseline_df.columns:
        data = baseline_df[col].dropna()
        if len(data) > 0:
            std = data.std()
            mad = stats.median_abs_deviation(data, scale='normal')
            print(f"  {method}:")
            print(f"    σ (std dev) = {std:.4f}°")
            print(f"    MAD = {mad:.4f}°")

# Per-method noise level - X POSITION
print("\n3. PER-METHOD NOISE - X POSITION (Standard Deviation in Baseline)")
for method, col in [('FFT', 'fft_x'), ('Sonar', 'sonar_x'), ('DVL', 'dvl_x')]:
    if col in baseline_df.columns:
        data = baseline_df[col].dropna()
        if len(data) > 0:
            std = data.std()
            mad = stats.median_abs_deviation(data, scale='normal')
            print(f"  {method}:")
            print(f"    σ (std dev) = {std:.4f} m")
            print(f"    MAD = {mad:.4f} m")

# Per-method noise level - Y POSITION
print("\n4. PER-METHOD NOISE - Y POSITION (Standard Deviation in Baseline)")
for method, col in [('FFT', 'fft_y'), ('Sonar', 'sonar_y'), ('DVL', 'dvl_y')]:
    if col in baseline_df.columns:
        data = baseline_df[col].dropna()
        if len(data) > 0:
            std = data.std()
            mad = stats.median_abs_deviation(data, scale='normal')
            print(f"  {method}:")
            print(f"    σ (std dev) = {std:.4f} m")
            print(f"    MAD = {mad:.4f} m")

# Pairwise biases - DISTANCE
print("\n5. PAIRWISE BIASES AND AGREEMENT - DISTANCE")
pairs = [
    ('FFT', 'DVL', 'diff_fft_nav'),
    ('Sonar', 'DVL', 'diff_sonar_nav'),
    ('FFT', 'Sonar', 'diff_fft_sonar')
]

for method1, method2, diff_col in pairs:
    if diff_col in baseline_df.columns:
        data = baseline_df[diff_col].dropna()
        if len(data) > 0:
            bias = data.mean()
            std = data.std()
            print(f"  {method1} vs {method2}:")
            print(f"    Bias (mean diff) = {bias:+.4f} m")
            print(f"    Std of diff = {std:.4f} m")
            print(f"    95% agreement = ±{1.96*std:.4f} m")

# Pairwise biases - PITCH
print("\n6. PAIRWISE BIASES AND AGREEMENT - PITCH")
pitch_pairs = [
    ('FFT', 'DVL', 'diff_pitch_fft_nav'),
    ('Sonar', 'DVL', 'diff_pitch_sonar_nav'),
    ('FFT', 'Sonar', 'diff_pitch_fft_sonar')
]

for method1, method2, diff_col in pitch_pairs:
    if diff_col in baseline_df.columns:
        data = baseline_df[diff_col].dropna()
        if len(data) > 0:
            bias = data.mean()
            std = data.std()
            print(f"  {method1} vs {method2}:")
            print(f"    Bias (mean diff) = {bias:+.4f}°")
            print(f"    Std of diff = {std:.4f}°")
            print(f"    95% agreement = ±{1.96*std:.4f}°")

# Pairwise biases - X POSITION
print("\n7. PAIRWISE BIASES AND AGREEMENT - X POSITION")
x_pairs = [
    ('FFT', 'DVL', 'diff_x_fft_nav'),
    ('Sonar', 'DVL', 'diff_x_sonar_nav'),
    ('FFT', 'Sonar', 'diff_x_fft_sonar')
]

for method1, method2, diff_col in x_pairs:
    if diff_col in baseline_df.columns:
        data = baseline_df[diff_col].dropna()
        if len(data) > 0:
            bias = data.mean()
            std = data.std()
            print(f"  {method1} vs {method2}:")
            print(f"    Bias (mean diff) = {bias:+.4f} m")
            print(f"    Std of diff = {std:.4f} m")
            print(f"    95% agreement = ±{1.96*std:.4f} m")

# Pairwise biases - Y POSITION
print("\n8. PAIRWISE BIASES AND AGREEMENT - Y POSITION")
y_pairs = [
    ('FFT', 'DVL', 'diff_y_fft_nav'),
    ('Sonar', 'DVL', 'diff_y_sonar_nav'),
    ('FFT', 'Sonar', 'diff_y_fft_sonar')
]

for method1, method2, diff_col in y_pairs:
    if diff_col in baseline_df.columns:
        data = baseline_df[diff_col].dropna()
        if len(data) > 0:
            bias = data.mean()
            std = data.std()
            print(f"  {method1} vs {method2}:")
            print(f"    Bias (mean diff) = {bias:+.4f} m")
            print(f"    Std of diff = {std:.4f} m")
            print(f"    95% agreement = ±{1.96*std:.4f} m")

=== BASELINE STATISTICS (776 samples) ===

1. PER-METHOD NOISE - DISTANCE (Standard Deviation in Baseline)
  FFT:
    σ (std dev) = 0.1067 m
    MAD = 0.1333 m
  Sonar:
    σ (std dev) = 0.0922 m
    MAD = 0.0771 m
  DVL:
    σ (std dev) = 0.1021 m
    MAD = 0.1186 m

2. PER-METHOD NOISE - PITCH (Standard Deviation in Baseline)
  FFT:
    σ (std dev) = 5.5465°
    MAD = 6.0436°
  Sonar:
    σ (std dev) = 8.4587°
    MAD = 9.3021°
  DVL:
    σ (std dev) = 8.6895°
    MAD = 10.2510°

3. PER-METHOD NOISE - X POSITION (Standard Deviation in Baseline)
  FFT:
    σ (std dev) = 0.1053 m
    MAD = 0.1286 m
  Sonar:
    σ (std dev) = 0.0918 m
    MAD = 0.0828 m
  DVL:
    σ (std dev) = 0.0986 m
    MAD = 0.1261 m

4. PER-METHOD NOISE - Y POSITION (Standard Deviation in Baseline)
  FFT:
    σ (std dev) = 0.1432 m
    MAD = 0.1470 m
  Sonar:
    σ (std dev) = 0.2037 m
    MAD = 0.2439 m
  DVL:
    σ (std dev) = 0.2192 m
    MAD = 0.2646 m

5. PAIRWISE BIASES AND AGREEMENT - DISTANCE
  FFT vs DVL:

In [202]:
# Linear relationships
print("\n9. LINEAR RELATIONSHIPS (in baseline)")

# Distance: FFT vs DVL
if 'fft_distance_m' in baseline_df.columns and 'nav_distance_m' in baseline_df.columns:
    valid = baseline_df[['fft_distance_m', 'nav_distance_m']].dropna()
    if len(valid) > 10:
        X = valid['nav_distance_m'].values.reshape(-1, 1)
        y = valid['fft_distance_m'].values
        reg = LinearRegression().fit(X, y)
        r2 = reg.score(X, y)
        print(f"  Distance - FFT vs DVL:")
        print(f"    fft_distance = {reg.intercept_:.4f} + {reg.coef_[0]:.4f} × dvl_distance")
        print(f"    R² = {r2:.4f}")

# Distance: Sonar vs DVL
if 'sonar_distance_m' in baseline_df.columns and 'nav_distance_m' in baseline_df.columns:
    valid = baseline_df[['sonar_distance_m', 'nav_distance_m']].dropna()
    if len(valid) > 10:
        X = valid['nav_distance_m'].values.reshape(-1, 1)
        y = valid['sonar_distance_m'].values
        reg = LinearRegression().fit(X, y)
        r2 = reg.score(X, y)
        print(f"  Distance - Sonar vs DVL:")
        print(f"    sonar_distance = {reg.intercept_:.4f} + {reg.coef_[0]:.4f} × dvl_distance")
        print(f"    R² = {r2:.4f}")

# Pitch: FFT vs DVL
if 'fft_pitch_deg' in baseline_df.columns and 'nav_pitch_deg' in baseline_df.columns:
    valid = baseline_df[['fft_pitch_deg', 'nav_pitch_deg']].dropna()
    if len(valid) > 10:
        X = valid['nav_pitch_deg'].values.reshape(-1, 1)
        y = valid['fft_pitch_deg'].values
        reg = LinearRegression().fit(X, y)
        r2 = reg.score(X, y)
        print(f"  Pitch - FFT vs DVL:")
        print(f"    fft_pitch = {reg.intercept_:.4f} + {reg.coef_[0]:.4f} × dvl_pitch")
        print(f"    R² = {r2:.4f}")

# Pitch: Sonar vs DVL
if 'sonar_pitch_deg' in baseline_df.columns and 'nav_pitch_deg' in baseline_df.columns:
    valid = baseline_df[['sonar_pitch_deg', 'nav_pitch_deg']].dropna()
    if len(valid) > 10:
        X = valid['nav_pitch_deg'].values.reshape(-1, 1)
        y = valid['sonar_pitch_deg'].values
        reg = LinearRegression().fit(X, y)
        r2 = reg.score(X, y)
        print(f"  Pitch - Sonar vs DVL:")
        print(f"    sonar_pitch = {reg.intercept_:.4f} + {reg.coef_[0]:.4f} × dvl_pitch")
        print(f"    R² = {r2:.4f}")


9. LINEAR RELATIONSHIPS (in baseline)
  Distance - FFT vs DVL:
    fft_distance = 0.2593 + 0.8456 × dvl_distance
    R² = 0.6537
  Distance - Sonar vs DVL:
    sonar_distance = 0.3389 + 0.7304 × dvl_distance
    R² = 0.6534
  Pitch - FFT vs DVL:
    fft_pitch = 3.7262 + 0.0332 × dvl_pitch
    R² = 0.0027
  Pitch - Sonar vs DVL:
    sonar_pitch = -1.1702 + 0.8566 × dvl_pitch
    R² = 0.7743


## 6. Outlier Detection Across Full Time Series

Identify spikes and anomalies using robust statistics.

In [203]:
print("=== OUTLIER DETECTION ===\n")

# Compute smooth baselines (rolling median)
smooth_window = int(2 * window_samples)  # Longer window for baseline

for method, col in [('FFT', 'fft_distance_m'), ('Sonar', 'sonar_distance_m'), ('DVL', 'nav_distance_m')]:
    if col in df.columns:
        df[f'{col}_baseline'] = df[col].rolling(smooth_window, center=True, min_periods=1).median()
        df[f'{col}_residual'] = df[col] - df[f'{col}_baseline']
        
        # Compute MAD-based outlier threshold
        residuals = df[f'{col}_residual'].dropna()
        if len(residuals) > 0:
            mad = stats.median_abs_deviation(residuals, scale='normal')
            threshold = OUTLIER_K * mad
            df[f'{col}_outlier'] = (df[f'{col}_residual'].abs() > threshold)
            
            n_outliers = df[f'{col}_outlier'].sum()
            outlier_rate = 100 * n_outliers / len(df)
            
            print(f"{method}:")
            print(f"  MAD = {mad:.4f} m")
            print(f"  Threshold = {threshold:.4f} m ({OUTLIER_K}× MAD)")
            print(f"  Outliers: {n_outliers} / {len(df)} ({outlier_rate:.2f}%)")
            
            if n_outliers > 0:
                outlier_residuals = df.loc[df[f'{col}_outlier'], f'{col}_residual']
                print(f"  Outlier magnitude: mean={outlier_residuals.abs().mean():.4f} m, "
                      f"max={outlier_residuals.abs().max():.4f} m")

=== OUTLIER DETECTION ===

FFT:
  MAD = 0.1143 m
  Threshold = 0.4002 m (3.5× MAD)
  Outliers: 487 / 2744 (17.75%)
  Outlier magnitude: mean=1.4903 m, max=18.7232 m
Sonar:
  MAD = 0.0627 m
  Threshold = 0.2195 m (3.5× MAD)
  Outliers: 7 / 2744 (0.26%)
  Outlier magnitude: mean=0.2346 m, max=0.2367 m
DVL:
  MAD = 0.0445 m
  Threshold = 0.1557 m (3.5× MAD)
  Outliers: 408 / 2744 (14.87%)
  Outlier magnitude: mean=0.2709 m, max=0.9300 m


In [204]:
print("\n=== FAILURE CORRELATION ===\n")

# Get outlier masks
outlier_cols = [c for c in df.columns if c.endswith('_outlier')]

if len(outlier_cols) >= 2:
    print("1. INDIVIDUAL OUTLIER RATES")
    for col in outlier_cols:
        # Extract method name and map to display name
        method_raw = col.replace('_distance_m_outlier', '').replace('_pitch_deg_outlier', '').replace('_x_outlier', '').replace('_y_outlier', '').replace('_', ' ').upper()
        # Map NAV to DVL for consistency
        method = 'DVL' if method_raw == 'NAV' else method_raw
        rate = 100 * df[col].sum() / len(df)
        print(f"  P({method} outlier) = {rate:.2f}%")
    
    print("\n2. CO-OCCURRENCE ANALYSIS")
    
    # Pairwise Jaccard indices
    from itertools import combinations
    for col1, col2 in combinations(outlier_cols, 2):
        method1_raw = col1.split('_')[0].upper()
        method2_raw = col2.split('_')[0].upper()
        
        # Map NAV to DVL for display
        method1 = 'DVL' if method1_raw == 'NAV' else method1_raw
        method2 = 'DVL' if method2_raw == 'NAV' else method2_raw
        
        both = (df[col1] & df[col2]).sum()
        either = (df[col1] | df[col2]).sum()
        
        if either > 0:
            jaccard = both / either
            print(f"  Jaccard({method1}, {method2}) = {jaccard:.3f}")
            
            # Conditional probabilities
            if df[col1].sum() > 0:
                p_2_given_1 = both / df[col1].sum()
                print(f"    P({method2} outlier | {method1} outlier) = {p_2_given_1:.3f}")
            if df[col2].sum() > 0:
                p_1_given_2 = both / df[col2].sum()
                print(f"    P({method1} outlier | {method2} outlier) = {p_1_given_2:.3f}")


=== FAILURE CORRELATION ===

1. INDIVIDUAL OUTLIER RATES
  P(FFT outlier) = 17.75%
  P(SONAR outlier) = 0.26%
  P(DVL outlier) = 14.87%

2. CO-OCCURRENCE ANALYSIS
  Jaccard(FFT, SONAR) = 0.014
    P(SONAR outlier | FFT outlier) = 0.014
    P(FFT outlier | SONAR outlier) = 1.000
  Jaccard(FFT, DVL) = 0.031
    P(DVL outlier | FFT outlier) = 0.055
    P(FFT outlier | DVL outlier) = 0.066
  Jaccard(SONAR, DVL) = 0.000
    P(DVL outlier | SONAR outlier) = 0.000
    P(SONAR outlier | DVL outlier) = 0.000


## 8. Visualization

Create comprehensive plots of all analyses.

In [205]:
# Plot 1: Full time series with baseline segments highlighted
fig1 = go.Figure()

for method, col, color in [('FFT', 'fft_distance_m', 'red'), 
                           ('Sonar', 'sonar_distance_m', 'blue'), 
                           ('DVL', 'nav_distance_m', 'green')]:
    if col in df.columns:
        fig1.add_trace(go.Scatter(
            x=df.index, y=df[col],
            mode='lines', name=method,
            line=dict(color=color, width=1)
        ))

# Highlight baseline segments
for seg in segments[:20]:  # First 20 segments
    fig1.add_vrect(
        x0=seg['start'], x1=seg['end'],
        fillcolor="lightgray", opacity=0.3,
        layer="below", line_width=0
    )

fig1.update_layout(
    title=f"Distance Time Series with Stable Baseline Segments: {TARGET_BAG}",
    xaxis_title="Time",
    yaxis_title="Distance (m)",
    height=500,
    hovermode='x unified'
)

fig1.show()
save_path = PLOTS_DIR / f"{TARGET_BAG}_full_timeseries.html"
fig1.write_html(str(save_path))
print(f"Saved: {save_path.name}")

Saved: 2024-08-20_13-57-42_full_timeseries.html


In [206]:
# Plot 1b: Stability metrics visualization
fig1b = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        'Distance with Stable Regions Highlighted',
        'Rolling Standard Deviation (Stability Metric)'
    ),
    vertical_spacing=0.15,
    row_heights=[0.6, 0.4]
)

# Top panel: Distance measurements with stable regions
for method, col, color in [('FFT', 'fft_distance_m', 'red'), 
                           ('Sonar', 'sonar_distance_m', 'blue'), 
                           ('DVL', 'nav_distance_m', 'green')]:
    if col in df.columns:
        fig1b.add_trace(go.Scatter(
            x=df.index, y=df[col],
            mode='lines', name=method,
            line=dict(color=color, width=1.5),
            showlegend=True
        ), row=1, col=1)

# Highlight stable regions on distance plot
for seg in segments:
    fig1b.add_vrect(
        x0=seg['start'], x1=seg['end'],
        fillcolor="lightgreen", opacity=0.2,
        layer="below", line_width=0,
        row=1, col=1
    )

# Bottom panel: Rolling standard deviations
for method, col, color in [('FFT', 'rolling_std_fft', 'red'), 
                           ('Sonar', 'rolling_std_sonar', 'blue'), 
                           ('DVL', 'rolling_std_nav', 'green')]:
    if col in df.columns:
        fig1b.add_trace(go.Scatter(
            x=df.index, y=df[col],
            mode='lines', name=f'{method} σ',
            line=dict(color=color, width=1.5, dash='dot'),
            showlegend=True
        ), row=2, col=1)

# Add stability threshold line
fig1b.add_hline(
    y=SIGMA_THRESH, 
    line_dash="dash", 
    line_color="black", 
    annotation_text=f"σ threshold = {SIGMA_THRESH} m",
    annotation_position="right",
    row=2, col=1
)

# Highlight stable regions on std plot
for seg in segments:
    fig1b.add_vrect(
        x0=seg['start'], x1=seg['end'],
        fillcolor="lightgreen", opacity=0.2,
        layer="below", line_width=0,
        row=2, col=1
    )

fig1b.update_xaxes(title_text="Time", row=2, col=1)
fig1b.update_yaxes(title_text="Distance (m)", row=1, col=1)
fig1b.update_yaxes(title_text="Rolling Std (m)", row=2, col=1)

fig1b.update_layout(
    title=f"Stability Analysis: {TARGET_BAG}",
    height=800,
    hovermode='x unified'
)

fig1b.show()
save_path = PLOTS_DIR / f"{TARGET_BAG}_stability_analysis.html"
fig1b.write_html(str(save_path))
print(f"Saved: {save_path.name}")

Saved: 2024-08-20_13-57-42_stability_analysis.html


In [207]:
# Plot 2: Pairwise differences
fig2 = make_subplots(rows=3, cols=1, subplot_titles=('FFT - DVL', 'Sonar - DVL', 'FFT - Sonar'))

diff_plots = [
    ('diff_fft_nav', 'red', 1),
    ('diff_sonar_nav', 'blue', 2),
    ('diff_fft_sonar', 'purple', 3)
]

for diff_col, color, row in diff_plots:
    if diff_col in df.columns:
        fig2.add_trace(go.Scatter(
            x=df.index, y=df[diff_col],
            mode='lines', name=diff_col.replace('diff_', '').replace('_', ' - ').upper(),
            line=dict(color=color, width=1)
        ), row=row, col=1)
        
        # Add zero line
        fig2.add_hline(y=0, line_dash="dash", line_color="gray", row=row, col=1)

fig2.update_xaxes(title_text="Time")
fig2.update_yaxes(title_text="Difference (m)")
fig2.update_layout(height=900, title_text=f"Pairwise Distance Differences: {TARGET_BAG}")

fig2.show()
save_path = PLOTS_DIR / f"{TARGET_BAG}_pairwise_differences.html"
fig2.write_html(str(save_path))
print(f"Saved: {save_path.name}")

Saved: 2024-08-20_13-57-42_pairwise_differences.html


In [208]:
# Plot 3: Outlier visualization
fig3 = go.Figure()

for method, col, outlier_col, color in [
    ('FFT', 'fft_distance_m', 'fft_distance_m_outlier', 'red'),
    ('Sonar', 'sonar_distance_m', 'sonar_distance_m_outlier', 'blue'),
    ('DVL', 'nav_distance_m', 'nav_distance_m_outlier', 'green')
]:
    if col in df.columns and outlier_col in df.columns:
        # Normal points
        normal_data = df[~df[outlier_col]]
        fig3.add_trace(go.Scatter(
            x=normal_data.index, y=normal_data[col],
            mode='lines', name=f'{method}',
            line=dict(color=color, width=1)
        ))
        
        # Outliers - smaller markers with transparency for overlap visibility
        outlier_data = df[df[outlier_col]]
        if len(outlier_data) > 0:
            fig3.add_trace(go.Scatter(
                x=outlier_data.index, y=outlier_data[col],
                mode='markers', name=f'{method} outliers',
                marker=dict(color=color, size=4, symbol='circle', 
                           opacity=0.7, line=dict(width=1, color='white'))
            ))

fig3.update_layout(
    title=f"Outlier Detection ({OUTLIER_K}× MAD): {TARGET_BAG}",
    xaxis_title="Time",
    yaxis_title="Distance (m)",
    height=500,
    hovermode='x unified'
)

fig3.show()
save_path = PLOTS_DIR / f"{TARGET_BAG}_outliers.html"
fig3.write_html(str(save_path))
print(f"Saved: {save_path.name}")

Saved: 2024-08-20_13-57-42_outliers.html


In [209]:
print("\n" + "="*70)
print("ANALYSIS SUMMARY")
print("="*70)

print(f"\nDataset: {TARGET_BAG}")
print(f"Duration: {(df.index[-1] - df.index[0]).total_seconds():.1f} s")
print(f"Samples: {len(df)}")
print(f"Available systems: {', '.join(available_systems)}")

print(f"\nStable baseline segments: {len(segments)}")
if len(segments) > 0:
    total_baseline_time = sum(s['duration_sec'] for s in segments)
    print(f"Total baseline time: {total_baseline_time:.1f} s ({100*total_baseline_time/(df.index[-1]-df.index[0]).total_seconds():.1f}%)")

print("\nBaseline noise levels:")
# Map display names to actual column names
noise_mapping = [
    ('FFT', 'fft_distance_m'),
    ('Sonar', 'sonar_distance_m'),
    ('DVL', 'nav_distance_m')  # Note: column is 'nav_distance_m', display as 'DVL'
]

for method_name, col in noise_mapping:
    if col in baseline_df.columns:
        data = baseline_df[col].dropna()
        if len(data) > 0:
            print(f"  {method_name}: σ = {data.std():.4f} m")

print("\nOutlier rates:")
for col in outlier_cols:
    method = col.split('_')[0].upper()
    # Map 'nav' to 'DVL' for display
    if method == 'NAV':
        method = 'DVL'
    rate = 100 * df[col].sum() / len(df)
    print(f"  {method}: {rate:.2f}%")

print(f"\nPlots saved to: {PLOTS_DIR}")
print("\n✓ Analysis complete")


ANALYSIS SUMMARY

Dataset: 2024-08-20_13-57-42
Duration: 79.9 s
Samples: 2744
Available systems: FFT, Sonar, DVL

Stable baseline segments: 3
Total baseline time: 22.7 s (28.4%)

Baseline noise levels:
  FFT: σ = 0.1067 m
  Sonar: σ = 0.0922 m
  DVL: σ = 0.1021 m

Outlier rates:
  FFT: 17.75%
  SONAR: 0.26%
  DVL: 14.87%

Plots saved to: /Volumes/LaCie/SOLAQUA/exports/plots

✓ Analysis complete


## 10. Generate Summary Report

In [210]:
# Generate comprehensive markdown summary report
import datetime

print("\n=== GENERATING SUMMARY REPORT ===\n")

# Save report to comparison data directory (where the CSV is)
COMPARISON_DATA_DIR.mkdir(parents=True, exist_ok=True)
report_path = COMPARISON_DATA_DIR / f"{TARGET_BAG}_analysis_summary.md"

with open(report_path, 'w') as f:
    # Header
    f.write(f"# Net Tracking Analysis Summary: {TARGET_BAG}\n\n")
    f.write(f"**Generated:** {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write("---\n\n")
    
    # Dataset info
    f.write("## Dataset Information\n\n")
    f.write(f"- **Bag ID:** {TARGET_BAG}\n")
    f.write(f"- **Duration:** {(df.index[-1] - df.index[0]).total_seconds():.1f} seconds\n")
    f.write(f"- **Total Samples:** {len(df)}\n")
    f.write(f"- **Sampling Rate:** ~{sampling_rate:.1f} Hz\n")
    f.write(f"- **Available Systems:** {', '.join(available_systems)}\n")
    f.write(f"- **Time Range:** {df.index[0]} to {df.index[-1]}\n\n")
    
    # Configuration
    f.write("## Analysis Configuration\n\n")
    if SMOOTHING_ALPHA is not None and SMOOTHING_ALPHA < 1.0:
        f.write(f"- **Smoothing:** Enabled (α={SMOOTHING_ALPHA:.2f})\n")
    else:
        f.write(f"- **Smoothing:** Disabled (raw data)\n")
    f.write(f"- **Rolling Window:** {ROLLING_WINDOW_SEC} s ({window_samples} samples)\n")
    f.write(f"- **Stability σ Threshold:** {SIGMA_THRESH} m\n")
    f.write(f"- **Stability Δ Threshold:** {DELTA_THRESH} m\n")
    f.write(f"- **Min Segment Length:** {MIN_SEGMENT_SEC} s\n")
    f.write(f"- **Outlier Threshold:** {OUTLIER_K} × MAD\n\n")
    
    # Stable baseline segments
    f.write("## Stable Baseline Segments\n\n")
    if len(segments) > 0:
        total_baseline_time = sum(s['duration_sec'] for s in segments)
        baseline_pct = 100 * total_baseline_time / (df.index[-1]-df.index[0]).total_seconds()
        f.write(f"- **Total Segments:** {len(segments)}\n")
        f.write(f"- **Total Baseline Time:** {total_baseline_time:.1f} s ({baseline_pct:.1f}%)\n")
        f.write(f"- **Average Segment Duration:** {total_baseline_time/len(segments):.1f} s\n\n")
    else:
        f.write("No stable baseline segments identified.\n\n")
    
    # Baseline noise levels
    f.write("## Baseline Noise Levels (Standard Deviation)\n\n")
    f.write("### Distance\n\n")
    f.write("| Method | σ (m) | MAD (m) |\n")
    f.write("|--------|-------|----------|\n")
    for method, col in [('FFT', 'fft_distance_m'), ('Sonar', 'sonar_distance_m'), ('DVL', 'nav_distance_m')]:
        if col in baseline_df.columns:
            data = baseline_df[col].dropna()
            if len(data) > 0:
                std = data.std()
                mad = stats.median_abs_deviation(data, scale='normal')
                f.write(f"| {method} | {std:.4f} | {mad:.4f} |\n")
    
    f.write("\n### Pitch\n\n")
    f.write("| Method | σ (°) | MAD (°) |\n")
    f.write("|--------|-------|----------|\n")
    for method, col in [('FFT', 'fft_pitch_deg'), ('Sonar', 'sonar_pitch_deg'), ('DVL', 'nav_pitch_deg')]:
        if col in baseline_df.columns:
            data = baseline_df[col].dropna()
            if len(data) > 0:
                std = data.std()
                mad = stats.median_abs_deviation(data, scale='normal')
                f.write(f"| {method} | {std:.4f} | {mad:.4f} |\n")
    
    f.write("\n### X Position (Perpendicular to Net)\n\n")
    f.write("| Method | σ (m) | MAD (m) |\n")
    f.write("|--------|-------|----------|\n")
    for method, col in [('FFT', 'fft_x'), ('Sonar', 'sonar_x'), ('DVL', 'dvl_x')]:
        if col in baseline_df.columns:
            data = baseline_df[col].dropna()
            if len(data) > 0:
                std = data.std()
                mad = stats.median_abs_deviation(data, scale='normal')
                f.write(f"| {method} | {std:.4f} | {mad:.4f} |\n")
    
    f.write("\n### Y Position (Along Net)\n\n")
    f.write("| Method | σ (m) | MAD (m) |\n")
    f.write("|--------|-------|----------|\n")
    for method, col in [('FFT', 'fft_y'), ('Sonar', 'sonar_y'), ('DVL', 'dvl_y')]:
        if col in baseline_df.columns:
            data = baseline_df[col].dropna()
            if len(data) > 0:
                std = data.std()
                mad = stats.median_abs_deviation(data, scale='normal')
                f.write(f"| {method} | {std:.4f} | {mad:.4f} |\n")
    
    # Pairwise biases - Distance
    f.write("\n## Pairwise Biases and Agreement\n\n")
    f.write("### Distance\n\n")
    f.write("| Comparison | Bias (m) | Std (m) | 95% Agreement (m) |\n")
    f.write("|------------|----------|---------|-------------------|\n")
    for method1, method2, diff_col in [('FFT', 'DVL', 'diff_fft_nav'), 
                                       ('Sonar', 'DVL', 'diff_sonar_nav'), 
                                       ('FFT', 'Sonar', 'diff_fft_sonar')]:
        if diff_col in baseline_df.columns:
            data = baseline_df[diff_col].dropna()
            if len(data) > 0:
                bias = data.mean()
                std = data.std()
                agreement = 1.96 * std
                f.write(f"| {method1} vs {method2} | {bias:+.4f} | {std:.4f} | ±{agreement:.4f} |\n")
    
    # Pairwise biases - Pitch
    f.write("\n### Pitch\n\n")
    f.write("| Comparison | Bias (°) | Std (°) | 95% Agreement (°) |\n")
    f.write("|------------|----------|---------|-------------------|\n")
    for method1, method2, diff_col in [('FFT', 'DVL', 'diff_pitch_fft_nav'), 
                                       ('Sonar', 'DVL', 'diff_pitch_sonar_nav'), 
                                       ('FFT', 'Sonar', 'diff_pitch_fft_sonar')]:
        if diff_col in baseline_df.columns:
            data = baseline_df[diff_col].dropna()
            if len(data) > 0:
                bias = data.mean()
                std = data.std()
                agreement = 1.96 * std
                f.write(f"| {method1} vs {method2} | {bias:+.4f} | {std:.4f} | ±{agreement:.4f} |\n")
    
    # Outlier rates
    f.write("\n## Outlier Detection Results\n\n")
    f.write("| Method | Outlier Rate (%) | Count |\n")
    f.write("|--------|------------------|-------|\n")
    for col in outlier_cols:
        method = col.split('_')[0].upper()
        if method == 'NAV':
            method = 'DVL'
        count = df[col].sum()
        rate = 100 * count / len(df)
        f.write(f"| {method} | {rate:.2f} | {count} |\n")
    
    # Failure correlation
    f.write("\n## Failure Correlation Analysis\n\n")
    f.write("### Co-occurrence (Jaccard Index)\n\n")
    f.write("| System Pair | Jaccard | P(A\|B) | P(B\|A) |\n")
    f.write("|-------------|---------|---------|----------|\n")
    
    from itertools import combinations
    for col1, col2 in combinations(outlier_cols, 2):
        method1_raw = col1.split('_')[0].upper()
        method2_raw = col2.split('_')[0].upper()
        method1 = 'DVL' if method1_raw == 'NAV' else method1_raw
        method2 = 'DVL' if method2_raw == 'NAV' else method2_raw
        
        both = (df[col1] & df[col2]).sum()
        either = (df[col1] | df[col2]).sum()
        
        if either > 0:
            jaccard = both / either
            p_2_given_1 = both / df[col1].sum() if df[col1].sum() > 0 else 0
            p_1_given_2 = both / df[col2].sum() if df[col2].sum() > 0 else 0
            f.write(f"| {method1}-{method2} | {jaccard:.3f} | {p_2_given_1:.3f} | {p_1_given_2:.3f} |\n")
    
    # Generated plots
    f.write("\n## Generated Plots\n\n")
    f.write("All plots saved to: `{}`\n\n".format(PLOTS_DIR))
    f.write("- Distance Time Series with Stable Segments\n")
    f.write("- Stability Analysis (Rolling Std)\n")
    f.write("- Pairwise Distance Differences\n")
    f.write("- Outlier Detection Visualization\n")
    f.write("- XY Trajectory Comparisons (if available)\n")
    
    # Footer
    f.write("\n---\n\n")
    f.write("*Generated by `06_full_net_analysis.ipynb`*\n")

print(f"✓ Summary report saved: {report_path.name}")
print(f"  Location: {report_path}")


=== GENERATING SUMMARY REPORT ===

✓ Summary report saved: 2024-08-20_13-57-42_analysis_summary.md
  Location: /Volumes/LaCie/SOLAQUA/comparison_data/2024-08-20_13-57-42_analysis_summary.md
